In [1]:
!pip install pycocotools pandas pillow matplotlib torch torchvision tqdm ipython

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from pycocotools.coco import COCO
from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm  # Added for progress bars

In [ ]:
# Paths
data_folder = '/kaggle/input/moon-ans'
output_folder = '/kaggle/working/'
train_img_dir = os.path.join(data_folder, 'train')
valid_img_dir = os.path.join(data_folder, 'valid')
test_img_dir = os.path.join(data_folder, 'test')
train_json = os.path.join(data_folder, 'train', '_annotations.coco.json')
valid_json = os.path.join(data_folder, 'valid', '_annotations.coco.json')
test_json = os.path.join(data_folder, 'test', '_annotations.coco.json')

In [ ]:
# Custom COCO Dataset
class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, coco_json, transform=None):
        self.img_dir = img_dir
        self.coco = COCO(coco_json)
        self.transform = transform
        self.img_ids = self.coco.getImgIds()

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.img_dir, img_info['file_name'])
        img = Image.open(img_path).convert('RGB')
        
        img = transforms.ToTensor()(img)
        if self.transform:
            img = self.transform(img)
        
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        boxes = []
        for ann in anns:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])
        
        boxes = torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 4), dtype=torch.float32)
        labels = torch.ones(boxes.size(0), dtype=torch.int64) if boxes.size(0) > 0 else torch.zeros((0,), dtype=torch.int64)
        
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id])
        }
        return img, target, img_id, img_info['file_name']

In [ ]:
# Custom Collate Function
def custom_collate_fn(batch):
    imgs, targets, img_ids, file_names = zip(*batch)
    imgs = torch.stack(imgs, dim=0)
    return imgs, list(targets), list(img_ids), list(file_names)

# mAP Calculation
def calculate_mAP(pred_boxes, pred_scores, true_boxes, iou_threshold=0.5):
    if len(pred_boxes) == 0 or len(true_boxes) == 0:
        return 0.0
    ious = []
    for pred in pred_boxes:
        for true in true_boxes:
            iou = calculate_iou(pred, true)
            ious.append(iou)
    ious = np.array(ious).reshape(len(pred_boxes), len(true_boxes))
    tp = (ious >= iou_threshold).sum()
    precision = tp / len(pred_boxes) if len(pred_boxes) > 0 else 0
    return precision

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    xi1 = max(x1, x3)
    yi1 = max(y1, y3)
    xi2 = min(x2, x4)
    yi2 = min(y2, y4)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

In [ ]:


# Training Function
def train_model(model, train_loader, valid_loader, epochs=15):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    # Updated optimizer with momentum (via betas) and weight decay as per paper
    optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=0.0005)
    best_mAP = 0
    best_model_path = os.path.join(output_folder, 'faster_rcnn_best.pth')
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        valid_samples = 0
        # Add tqdm for training loop
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]', leave=False)
        for imgs, targets, _, _ in train_pbar:
            imgs = imgs.to(device)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(imgs, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            train_loss += losses.item()
            valid_samples += 1
            
            # Update progress bar with running loss
            train_pbar.set_postfix({'loss': f'{losses.item():.4f}'})
        
        # Validation
        model.eval()
        valid_mAP = 0
        valid_count = 0
        # Add tqdm for validation loop
        valid_pbar = tqdm(valid_loader, desc=f'Epoch {epoch+1}/{epochs} [Valid]', leave=False)
        with torch.no_grad():
            for imgs, targets, _, _ in valid_pbar:
                imgs = imgs.to(device)
                outputs = model(imgs)
                for i, output in enumerate(outputs):
                    true_boxes = targets[i]['boxes'].cpu().numpy()
                    if len(true_boxes) == 0:
                        continue
                    pred_boxes = output['boxes'].cpu().numpy()
                    pred_scores = output['scores'].cpu().numpy()
                    mask = pred_scores > 0.5
                    pred_boxes = pred_boxes[mask]
                    pred_scores = pred_scores[mask]
                    valid_mAP += calculate_mAP(pred_boxes, pred_scores, true_boxes)
                    valid_count += 1
        
        train_loss_avg = train_loss / valid_samples if valid_samples > 0 else 0
        valid_mAP_avg = valid_mAP / valid_count if valid_count > 0 else 0
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss_avg:.4f}, mAP@0.5: {valid_mAP_avg:.4f}')
        
        if valid_mAP_avg > best_mAP:
            best_mAP = valid_mAP_avg
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with mAP@0.5: {best_mAP:.4f}')
    
    torch.save(model.state_dict(), os.path.join(output_folder, 'faster_rcnn_final.pth'))


In [ ]:
# Prediction Function
def generate_predictions(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    predictions = []
    invalid_boxes = 0
    
    # Add tqdm for prediction loop
    test_pbar = tqdm(test_loader, desc='Predicting', leave=False)
    with torch.no_grad():
        for imgs, targets, img_ids, file_names in test_pbar:
            imgs = imgs.to(device)
            outputs = model(imgs)
            for i, (img_id, file_name, output) in enumerate(zip(img_ids, file_names, outputs)):
                pred_boxes = output['boxes'].cpu().numpy()
                pred_scores = output['scores'].cpu().numpy()
                mask = pred_scores > 0.5
                pred_boxes = pred_boxes[mask]
                
                for box in pred_boxes:
                    x_min, y_min, x_max, y_max = box
                    w = max(0, x_max - x_min)
                    h = max(0, y_max - y_min)
                    if w * h == 0:
                        invalid_boxes += 1
                        continue
                    diameter = np.sqrt(w * h)
                    predictions.append({
                        'image_id': img_id,
                        'file_name': file_name,
                        'x': x_min,
                        'y': y_min,
                        'width': w,
                        'height': h,
                        'diameter': diameter
                    })
    
    print(f"Generated {len(predictions)} predictions. Skipped {invalid_boxes} invalid boxes (w or h <= 0).")
    pd.DataFrame(predictions).to_csv(os.path.join(output_folder, 'crater_predictions.csv'), index=False)


In [2]:

# Main
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(640, scale=(0.8, 1.2)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_test_transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CocoDataset(train_img_dir, train_json, transform=train_transform)
valid_dataset = CocoDataset(valid_img_dir, valid_json, transform=valid_test_transform)
test_dataset = CocoDataset(test_img_dir, test_json, transform=valid_test_transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=custom_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=custom_collate_fn)

# Load Faster R-CNN with ResNet-50-FPN
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  # 1 class (crater) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

train_model(model, train_loader, valid_loader, epochs=15)
generate_predictions(model, test_loader)

print('Training complete. Models saved as faster_rcnn_best.pth and faster_rcnn_final.pth')
print('Predictions saved as crater_predictions.csv')

loading annotations into memory...
Done (t=0.41s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/15, Loss: 1.2653, mAP@0.5: 0.3295
Saved best model with mAP@0.5: 0.3295


Epoch 2/15, Loss: 1.3051, mAP@0.5: 0.6414
Saved best model with mAP@0.5: 0.6414


Epoch 3/15, Loss: 1.3102, mAP@0.5: 0.4542


...

Epoch 12/15, Loss: 1.3091, mAP@0.5: 0.6613


Epoch 13/15, Loss: 1.3104, mAP@0.5: 0.6691


Epoch 14/15, Loss: 1.3050, mAP@0.5: 0.6981


Epoch 15/15, Loss: 1.3062, mAP@0.5: 0.6492


Generated 703 predictions. Skipped 0 invalid boxes (w or h <= 0).
Training complete. Models saved as faster_rcnn_best.pth and faster_rcnn_final.pth
Predictions saved as crater_predictions.csv
